In [ ]:
def IntialiseEnv():
    nb_dir = os.path.split(os.getcwd())[0]
    if nb_dir not in sys.path:
        sys.path.append(nb_dir)

In [ ]:
## Import libraries need to be imported, Dont forget to update requirements.txt!
import os
import sys
IntialiseEnv()
import glob
import numpy as np
import cv2
import warnings
import pandas as pd 
import pickle
import import_ipynb
from Source_Code.Pre_Processing.Pre_Processing import FeatureExtraction

## Use GPU/ CPU Configuration 
Tensorflow version 2.0
Prompt to user if CPU/ GPU is in use with device name

In [ ]:
# Use GPU/CPU Configurations

import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

### Loading Images for German and Belgium

In [ ]:
## Read image in path
def readImage(path):
    img= cv2.imread(path)
    img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img= cv2.resize(img,(60,60))
    return img

In [ ]:
## Function to get csv path
def getCsvPath(train_path_dir):
     return os.path.join(train_path_dir,list(filter(lambda x: '.csv' in x, os.listdir(train_path_dir)))[0])

In [ ]:
# Function to get category directory and corresponding data frame
def getDirAndDataFrame(datasetDir,_dir):
    train_category_dir= os.path.join(datasetDir,_dir)
    train_csv_path= getCsvPath(train_category_dir)
    train_Data_frame = pd.read_csv(train_csv_path, delimiter=';')
    return train_category_dir, train_Data_frame

In [ ]:
test_dir='../Source_Code/DataSet/Testing_DataSet/Final_Test/Images/'
def LoadGermanImages():
    test_image_array=[]
   
    for _dir in os.listdir(test_dir):
        if _dir=='GT-final_test.test.csv':
            continue
        img = readImage(test_dir+_dir)
        test_image_array.append(img)
    print("Loaded German Images without feature extraction")   
    return  np.array(test_image_array)

In [ ]:
test_belgium_dataset_dir='../Source_Code/DataSet/BelgiumTSC_Testing/Testing/'
def LoadBelgiumImages():
    test_image_array=[]

    for _dir in os.listdir(test_belgium_dataset_dir):
        # Get Directory for train with certain class ID and its associated dataframe
        test_category_dir, test_data_frame= getDirAndDataFrame(test_belgium_dataset_dir, _dir)
        # Get ClassID for entire category
        if len(test_data_frame['ClassId']) == 0:
            continue

        for img_path in glob.glob(os.path.join(test_category_dir, '*.ppm')):
            img = readImage(img_path)
            test_image_array.append(img)
            
    test_image_array=np.stack(test_image_array, axis=0)
    print("Loaded Belgium Images without feature extraction")  
    return test_image_array

## Computation Avg. Processing Time

In [ ]:
def ComputeGermanAvgProcessingTime():
    test_image_array= LoadGermanImages()
    model = tf.keras.models.load_model('../Source_Code/Model/Trained_Models/German_Winning_99_69.h5')
    model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
  
    import time
    t1=time.time()
    test_images=[FeatureExtraction(test_image)[:,:,np.newaxis] for test_image in test_image_array]
    test_images=np.stack(test_images, axis=0)
    model.predict(test_images)
    t2=time.time()
    print("Average Processing time: ", ((t2-t1)/12630)*1000, " ms")

In [ ]:
def ComputeBelgiumAvgProcessingTime():
    test_image_array= LoadBelgiumImages()
    model = tf.keras.models.load_model('../Source_Code/Model/Trained_Models/Belgium_Winning_98_97.h5')
    model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
  
    import time
    t1=time.time()
    test_images=[FeatureExtraction(test_image)[:,:,np.newaxis] for test_image in test_image_array]
    test_images=np.stack(test_images, axis=0)
    model.predict(test_images)
    t2=time.time()
    print("Average Processing time: ", ((t2-t1)/2520)*1000, " ms")

In [ ]:
ComputeGermanAvgProcessingTime()

In [ ]:
ComputeBelgiumAvgProcessingTime()